In [22]:
import pandas as pd

url1 = 'https://www.ncei.noaa.gov/data/global-hourly/access/2023/91190022516.csv'

NOAA_Kahului_df = pd.read_csv(url1)


url2 = 'https://www.ncei.noaa.gov/data/global-hourly/access/2023/99738499999.csv'

NOAA_Moluoloe_df = pd.read_csv(url2)


#narrow down to just the 8th through the 11th

# Convert the 'DATE' column to datetime objects
NOAA_Kahului_df['DATE'] = pd.to_datetime(NOAA_Kahului_df['DATE'])
NOAA_Moluoloe_df['DATE'] = pd.to_datetime(NOAA_Moluoloe_df['DATE'])

# Define the start and end of the desired date range
start_date = '2023-08-08'
end_date = '2023-08-11'

# Filter the DataFrame to include only rows within the specified date range
mask = (NOAA_Kahului_df['DATE'] >= start_date) & (NOAA_Kahului_df['DATE'] <= end_date)
NOAA_Kahului_df = NOAA_Kahului_df.loc[mask]
mask = (NOAA_Moluoloe_df['DATE'] >= start_date) & (NOAA_Moluoloe_df['DATE'] <= end_date)
NOAA_Moluoloe_df = NOAA_Moluoloe_df.loc[mask]


#print(NOAA_Kahului_df.head())
#print(NOAA_Moluoloe_df.head())

#combine into one df, sort by date and time, drop flags we don't need
#columns_list1 = NOAA_Kahului_df.columns.tolist()
#columns_list2 = NOAA_Moluoloe_df.columns.tolist()
#print(columns_list1)
#print(columns_list2)

#flags to keep
# STATION, DATE, LATITUDE, LONGITUDE, ELEVATION, NAME, WND, TMP, DEW, SLP

# some useful info on the data
# Note that I ignore a lot of the quality control measures because, for the most part,
# I haven't seen many bad quality markers. So as to keep the model simple I thought it
# best we ignore them

# WND is broken down into 5 sections like 070,1,N,0134,1
#       section 070 is The angle, measured in a clockwise direction, between true
#               north and the direction from which the wind is blowing.
#       the section 1 is quality control that we should ignore for now
#       the section N is the code that denotes that character, lets also ignore this for now
#       the section 0134 is 9 WIND-OBSERVATION speed rate
#                     The rate of horizontal travel of air past a fixed point.
#                     MIN: 0000 MAX: 0900 UNITS: meters per second
#       the section 1 is another quality control measure that we should ignore for now
# so we can pull from WND the WND_ANGLE and WND_SPEED


# TMP is broken down into two sections similar to +0311, 1
#           the 0311 section is The temperature of the air.
#                 MIN: -0932 MAX: +0618 UNITS: Degrees Celsius
#           the 1 section is quality control which we should ignore for now
# so from TMP we can pull TMP_CELC

# DEW is broken down into two sections similar to +0200, 1
#            the +0200 section is The temperature to which a given parcel of air must
#                   be cooled at constant pressure and water vapor content in
#                   order for saturation to occur.
#                   MIN: -0982 MAX: +0368 UNITS: Degrees Celsius
#             the 1 section is quality control which we should ignore for now
# so from DEW we can pull DEW_TMP

#SLP is Sea Level Pressure is broken down into two sections similar to 10167,1
#           the 10167 section is 0-104 ATMOSPHERIC-PRESSURE-OBSERVATION sea level pressure
#                  The air pressure relative to Mean Sea Level (MSL).
#                  MIN: 08600 MAX: 10900 UNITS: Hectopascals
#            the 1 sections is quality control which we should ignore for now
# so from SLP we can pull SLP

#I'll combine the df's with just the flags to keep first, then I'll parse out the flags I want to parse out
columns_to_keep = ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP', 'DEW', 'SLP']

# Narrow down the DataFrame to the specified columns
NOAA_Kahului_df = NOAA_Kahului_df[columns_to_keep]
NOAA_Moluoloe_df = NOAA_Moluoloe_df[columns_to_keep]

NOAA_df = NOAA_Kahului_df.append(NOAA_Moluoloe_df, ignore_index=True)

#sort NOAA_df by date and look at the data
NOAA_df = NOAA_df.sort_values(by='DATE')
#print(NOAA_df.head())

#now parse out WND
wnd_components = NOAA_df['WND'].str.split(',', expand=True)
NOAA_df['WND_ANGLE'] = wnd_components[0]
NOAA_df['WND_SPEED'] = wnd_components[3]
NOAA_df['WND_ANGLE'] = pd.to_numeric(NOAA_df['WND_ANGLE'], errors='coerce')
NOAA_df['WND_SPEED'] = pd.to_numeric(NOAA_df['WND_SPEED'], errors='coerce')
#print(NOAA_df[['WND_ANGLE', 'WND_SPEED']].head())

#now parse out TMP
tmp_components = NOAA_df['TMP'].str.split(',', expand=True)
NOAA_df['TMP'] = tmp_components[0]
#print(NOAA_df[['TMP']].head())

#now parse out DEW
dew_components = NOAA_df['DEW'].str.split(',', expand=True)
NOAA_df['DEW_TMP'] = dew_components[0]
#print(NOAA_df[['DEW', 'DEW_TMP']].head())

#now parse out SLP
slp_components = NOAA_df['SLP'].str.split(',', expand=True)
NOAA_df['SLP'] = slp_components[0]
#print(NOAA_df[['SLP']].head())

#final formatting of the NOAA data
columns_to_keep = ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND_ANGLE','WND_SPEED', 'TMP', 'DEW_TMP', 'SLP']#
NOAA_df = NOAA_df[columns_to_keep]
print(NOAA_df.head())

        STATION                DATE  LATITUDE  LONGITUDE  ELEVATION  \
0   91190022516 2023-08-08 00:00:00  20.88871 -156.43453       14.0   
90  99738499999 2023-08-08 00:00:00  20.90000 -156.47000        4.0   
1   91190022516 2023-08-08 00:54:00  20.88871 -156.43453       14.0   
91  99738499999 2023-08-08 01:00:00  20.90000 -156.47000        4.0   
2   91190022516 2023-08-08 01:54:00  20.88871 -156.43453       14.0   

                      NAME  WND_ANGLE  WND_SPEED    TMP DEW_TMP    SLP  
0   KAHULUI AIRPORT, HI US         70        134  +0311   +0200  10167  
90         MOLUOLOE, HI US         80        144  +0276   +9999  10165  
1   KAHULUI AIRPORT, HI US         60        134  +0317   +0194  10165  
91         MOLUOLOE, HI US         80        113  +0272   +9999  10166  
2   KAHULUI AIRPORT, HI US         50        124  +0306   +0200  10166  
